## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Los Llanos De Aridane,ES,2021-04-25 23:37:06,28.6585,-17.9182,68.00,68,75,8.86
1,1,Ushuaia,AR,2021-04-25 23:44:20,-54.8000,-68.3000,50.00,71,75,24.16
2,2,Linxia,CN,2021-04-25 23:46:53,35.6003,103.2064,42.48,75,17,1.48
3,3,Sulangan,PH,2021-04-25 23:46:54,11.1373,123.7228,81.66,75,23,12.82
4,4,Mahon,ES,2021-04-25 23:46:54,39.8885,4.2658,59.00,77,0,17.27


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Vao,NC,2021-04-25 23:36:58,-22.6667,167.4833,74.50,69,82,15.23
8,8,Benguela,AO,2021-04-25 23:46:36,-12.5763,13.4055,78.28,84,50,1.43
15,15,Sibu,MY,2021-04-25 23:46:56,2.3000,111.8167,75.20,94,20,4.61
17,17,Port Alfred,ZA,2021-04-25 23:36:59,-33.5906,26.8910,70.63,56,100,11.83
25,25,Bowen,AU,2021-04-25 23:46:58,-20.0167,148.2333,75.20,73,90,11.50
39,39,Arraial Do Cabo,BR,2021-04-25 23:47:02,-22.9661,-42.0278,74.32,83,85,9.01
43,43,Saint-Philippe,RE,2021-04-25 23:47:03,-21.3585,55.7679,77.00,73,0,17.27
44,44,Rikitea,PF,2021-04-25 23:47:03,-23.1203,-134.9692,77.97,75,86,5.95
53,53,Cantanhede,BR,2021-04-25 23:47:05,-3.6333,-44.3767,74.30,96,59,3.00
67,67,Rayagada,IN,2021-04-25 23:47:09,19.1667,83.4167,76.73,70,8,2.93


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       156
City          156
Country       156
Date          156
Lat           156
Lng           156
Max Temp      156
Humidity      156
Cloudiness    156
Wind Speed    156
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_new_df = preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_new_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Vao,NC,74.50,-22.6667,167.4833,
8,Benguela,AO,78.28,-12.5763,13.4055,
15,Sibu,MY,75.20,2.3000,111.8167,
17,Port Alfred,ZA,70.63,-33.5906,26.8910,
25,Bowen,AU,75.20,-20.0167,148.2333,
39,Arraial Do Cabo,BR,74.32,-22.9661,-42.0278,
43,Saint-Philippe,RE,77.00,-21.3585,55.7679,
44,Rikitea,PF,77.97,-23.1203,-134.9692,
53,Cantanhede,BR,74.30,-3.6333,-44.3767,
67,Rayagada,IN,76.73,19.1667,83.4167,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Vao,NC,74.50,-22.6667,167.4833,Hotel Kou-Bugny
8,Benguela,AO,78.28,-12.5763,13.4055,Hotel Praia Morena
15,Sibu,MY,75.20,2.3000,111.8167,RH HOTEL (4 Star)
17,Port Alfred,ZA,70.63,-33.5906,26.8910,The Halyards Hotel
25,Bowen,AU,75.20,-20.0167,148.2333,Castle Motor Lodge Motel
...,...,...,...,...,...,...
703,Marawi,PH,79.00,7.9986,124.2928,Derogongan Residence
713,Raga,SS,72.34,8.4596,25.6780,
717,Springdale,US,73.40,36.1867,-94.1288,Springdale Inn
718,Santa Isabel Do Rio Negro,BR,76.15,-0.4139,-65.0192,Hotel GSA


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))